In [10]:
import streamlit as st
# from langchain import LangChain
import PyPDF2
# import pypdf2 as PyPDF2
from io import StringIO
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import  MessagesPlaceholder, PromptTemplate, ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
import os

In [11]:

ai_avatar  = "🤖"
user_avatar = "💬"
def read_pdf(file):
    
    # stringio = StringIO(file.getvalue().decode("utf-8"))
     # To read file as bytes:
    # bytes_data = file.getvalue()
    # To convert to a string based IO:
    # stringio = StringIO(file.getvalue())#.decode("utf-8"))

    # pdf_file_obj = open(file.name, 'rb')
    # pdf_file_obj = open(file.getvalue(),'rb')
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        # page_obj = pdf_reader.getPage(page_num)
        page_obj = pdf_reader.pages[page_num]
        text += page_obj.extract_text()
    # pdf_file_obj.close()
    return text

In [12]:
def get_template_string():
    
    return """You are a a specialized career coach for the data science and analytics sector, focused on delivering tailored, concise job application advice. 
    You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.
    When analyzing a resume vs. a job listing, start by categorizing a user's fit for a job as 'perfect,' 'great,' 'good,' or 'non-ideal' based on the resume and job listing comparison before going into detail.
    You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable, with the goal of enhancing user confidence and aiding their career progression.
    """

In [13]:
model_type = st.sidebar.radio("GPT Model", options=["gpt-3.5-turbo-0125", "gpt-3.5-turbo-instruct","gpt-4-turbo-preview"],
                              index=0)

def get_llm(model_type=model_type, temperature=0.1, #verbose=False,
             template_string_func=get_template_string):

    ## get prompt string
    system_prompt = template_string_func()
    
    prompt_template =system_prompt+"""
    Current conversation:
    {history}
    Human: {input}
    AI:"""
    prompt = ChatPromptTemplate.from_template(prompt_template)
    llm = ChatOpenAI(temperature=.1, model=model_type)
    llm_chain = ConversationChain(prompt=prompt, llm=llm, memory=ConversationBufferWindowMemory(),#ConversationBufferMemory(memory_key="history"),
                                  verbose=True, output_key="response")#,#callbacks=callbacks)
    
    # llm_chain.memory.chat_memory.add_sy (template)
    return llm_chain


In [14]:

def fake_streaming(response):
    import time
    for word in response.split(" "):
        yield word + " "
        time.sleep(.05)		
        
            
    
## For steramlit try this as raw code, not a function
def print_history(llm_chain):
    # Simulate streaming for final message

    session_state_messages = llm_chain.memory.buffer_as_messages
    for msg in session_state_messages:#[:-1]:
        if isinstance(msg, AIMessage):
            # notebook
            print(f"Assistant: {msg.content}")
            # streamlit
            st.chat_message("assistant", avatar=ai_avatar).write(msg.content)
        
        elif isinstance(msg, HumanMessage):
            # notebook
            print(f"User: {msg.content}")
            # streamlit
            st.chat_message("user", avatar=user_avatar).write(msg.content)
        print()




def get_context_strings(context_resume=None, context_job=None,):
    if context_resume is None:
        context_resume = st.session_state.resume_text
    
    if context_job is None:
        context_job = st.session_state.job_listing_text
    # task_prompt_dict = get_task_options(options_only=False)
    # system_prompt = task_prompt_dict[selected_task]
    
    # template_assistant = "You are a helpful assistant data scientist who uses NLP analysis of customer reviews to inform business-decision-making:"
    # product_template = f" Assume all user questions are asking about the content in the user reviews. Note the product metadata is:\n```{product_string}```\n\n"
    # template_starter = get_template_string()
    context = f"\nHere is my resume: \n\n-------------\n {context_resume}.\n\n Here is the job listing:\n\n-------------\n{context_job}.\n\n"
    # context += f"Use the {context_type} first before using the retrieved documents."
    # template_assistant=template_starter+ context
    return context


def get_task_options(options_only=False):
    task_prompt_dict= {
        # "Summary of Customer Sentiment":'Provide a summary list of what 1-star reviews did not like and a summary of what did 5-star reviews liked.',
                   'Compare Resume vs. Job':'How does my resume compare against this job listing? Provide a detailed break down list comparing my resume vs. the listing. Highlight any weaknesses.',
                   'Craft Covert Letter':'Write a cover letter for this job. Use a non-traditional opening sentence that is very enthusiastic about the job and memorable (you can even use !). The reamining portions of the letter should be high quality and speak to my experience.',
                   "Interview Prep":'Help me prepare for an interview. Start with a question that I should expect in the interview.',
                   "Resume Analysis":'Analyze my resume. Start with a summary of my resume.',}

    if options_only:
        return list(task_prompt_dict.keys())
    else:
        return task_prompt_dict


In [15]:
task_options = get_task_options(options_only=False)
task_options

{'Compare Resume vs. Job': 'How does my resume compare against this job listing? Provide a detailed break down list comparing my resume vs. the listing. Highlight any weaknesses.',
 'Craft Covert Letter': 'Write a cover letter for this job. Use a non-traditional opening sentence that is very enthusiastic about the job and memorable (you can even use !). The reamining portions of the letter should be high quality and speak to my experience.',
 'Interview Prep': 'Help me prepare for an interview. Start with a question that I should expect in the interview.',
 'Resume Analysis': 'Analyze my resume. Start with a summary of my resume.'}

### Rethinking the Prompt

In [16]:
selected_task = 'Compare Resume vs. Job'
prompt_text =  task_options[selected_task]
print(prompt_text)


How does my resume compare against this job listing? Provide a detailed break down list comparing my resume vs. the listing. Highlight any weaknesses.


In [17]:
resume = read_pdf('data/James Irving Resume 2024.pdf')
print(resume[:1000])

JAMES M. IRVING, PH.D. 8222 Spadderdock Way, Laurel, MD, 20724 | (518) 322-6750 | james.irving.phd@gmail.com |  LINKEDIN: james-irving-phd | GITHUB: https://github.com/jirvingphd  SUMMARY Innovative and enterprising data scientist. Experienced in applying advanced data science techniques to real-world problems, including statistical modeling, machine learning, and data visualization. Skilled at extracting valuable insights from complex datasets and supporting data-driven decision-making. Adept Python programmer with the demonstrated ability to rapidly master and apply new technologies. A highly analytical professional committed to continuous learning and leveraging the value of data to innovate and transform. Former neuroscience researcher with a solid foundation in science, experimental design, and cognitive neuroscience. COMPETENCIES • Data Analysis • Statistical Modeling • Machine Learning • Data Visualization • Experimental Design • Quantitative Research Methods • Time Series Analy

In [18]:
job = read_pdf('data/Data Scientist - Focus School Software _ Remote.pdf')
print(job[:1000])

4/4/24, 2:48 PM Data Scientist - Focus School Software | Remote
https://www.simplyhired.com/job/lxgFxvpAXhH6zOMvh9G6pzooFKCORCgkfSudwEuQZAQjF7dORlMCSw?from=smart-apply 1/4Post Jobs
D t  Scientist
Focus School Softw re
Remote
Job Det ils
Full-time
$91,751.28 - $110,496.16   ye r
2 d ys   o
Beneﬁts
He lth s vin s  ccount Dis bility insur nce He lth insur nce
Dent l insur nce 401(k) Flexible spendin   ccount P id time off
Vision insur nce 401(k) m tchin  Life insur nce
Qu liﬁc tions
Power BI Writin  skills D t   n lysis skills Rese rch 3 ye rs
SQL Re ression  n lysis 2 ye rs Communic tion skills An lytics
Full Job Description
Focus is lookin  for ener etic, cre tive,  nd experienced c ndid tes who c n help us
succeed in our mission to serve  n incre sin  number of districts  cross the n tion. We
keep our clients’ needs  t the forefront of everythin  we do. As   comp ny who
underst nds the h rd work of tod y’s educ tors, Focus looks for employees who sh re our
v lues tow rds educ tion.
Foc

In [19]:
context = get_context_strings(resume,job)
context


'\nHere is my resume: \n\n-------------\n JAMES M. IRVING, PH.D. 8222 Spadderdock Way, Laurel, MD, 20724 | (518) 322-6750 | james.irving.phd@gmail.com |  LINKEDIN: james-irving-phd | GITHUB: https://github.com/jirvingphd  SUMMARY Innovative and enterprising data scientist. Experienced in applying advanced data science techniques to real-world problems, including statistical modeling, machine learning, and data visualization. Skilled at extracting valuable insights from complex datasets and supporting data-driven decision-making. Adept Python programmer with the demonstrated ability to rapidly master and apply new technologies. A highly analytical professional committed to continuous learning and leveraging the value of data to innovate and transform. Former neuroscience researcher with a solid foundation in science, experimental design, and cognitive neuroscience. COMPETENCIES • Data Analysis • Statistical Modeling • Machine Learning • Data Visualization • Experimental Design • Quantit

In [20]:
# # system_prompt = 'How does my resume compare against this job listing? Provide a detailed break down list comparing my resume vs. the listing. Highlight any weaknesses.',
# def get_template_string():
    
#     return """You are a a specialized career coach for the data science and analytics sector, focused on delivering tailored, concise job application advice. 
#     You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.
#     When analyzing a resume vs. a job listing, start by categorizing a user's fit for a job as 'perfect,' 'great,' 'good,' or 'non-ideal' based on the resume and job listing comparison before going into detail.
#     You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable, with the goal of enhancing user confidence and aiding their career progression.
#     """
    
system_prompt = (" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice. " 
" You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements. "
" You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable, "
" with the goal of aiding their career progression. ")
# final_prompt = prompt_text + context
# print(final_prompt)
system_prompt



" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice.  You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.  You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable,  with the goal of aiding their career progression. "

In [21]:
#Question: include context in the system prompt or not?
context = "\nUse the following context, if provided, to help answer the questions:\n\nHere is my resume:\n-------------\n {resume}\n\n Here is the job listing:\n-------------\n{job}\n\n"
final_promp_str = system_prompt + context
print(final_promp_str)


 You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice.  You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.  You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable,  with the goal of aiding their career progression. 
Use the following context, if provided, to help answer the questions:

Here is my resume:
-------------
 {resume}

 Here is the job listing:
-------------
{job}




In [22]:
final_promp_str =system_prompt+ context + """
    Current conversation:
    {history}
    Human: {input}
    AI:"""
final_prompt_template = ChatPromptTemplate.from_template(final_promp_str)
final_prompt_template


ChatPromptTemplate(input_variables=['history', 'input', 'job', 'resume', 'sector'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['history', 'input', 'job', 'resume', 'sector'], template=" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice.  You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.  You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable,  with the goal of aiding their career progression. \nUse the following context, if provided, to help answer the questions:\n\nHere is my resume:\n-------------\n {resume}\n\n Here is the job listing:\n-------------\n{job}\n\n\n    Current conversation:\n    {history}\n    Human: {input}\n    AI:"))])

In [23]:
def get_system_prompt_str():
    """Helper function for get_prompt_template. New v2.0"""
    system_prompt = (" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice. " 
    " You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements. "
    " You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable, "
    " with the goal of aiding their career progression. Ask the user for their resume and job listing if not provided and they are needed to asnwer .")
    context = "\nUse the following context, if provided, to help answer the questions:\n\nHere is my resume:\n-------------\n {resume}\n\n Here is the job listing:\n-------------\n{job}\n\n "
    
    
    return system_prompt + context

def get_prompt_template(system_prompt):
    # system_prompt = get_system_prompt_str()
    final_promp_str = system_prompt + """
        Current conversation:
        {history}
        Human: {input}
        AI:"""
        
    final_prompt_template = ChatPromptTemplate.from_template(final_promp_str)
    return final_prompt_template


prompt = get_prompt_template(get_system_prompt_str())
prompt


ChatPromptTemplate(input_variables=['history', 'input', 'job', 'resume', 'sector'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['history', 'input', 'job', 'resume', 'sector'], template=" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice.  You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.  You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable,  with the goal of aiding their career progression. Ask the user for their resume and job listing if not provided and they are needed to asnwer .\nUse the following context, if provided, to help answer the questions:\n\nHere is my resume:\n-------------\n {resume}\n\n Here is the job listing:\n-------------\n{job}\n\n \n        Current conversation:\n        {history}\n        Huma

In [24]:
prompt.input_variables

['history', 'input', 'job', 'resume', 'sector']

In [25]:
# Filling in partial prompt
sector = "data science and analytics"
resume = read_pdf('data/James Irving Resume 2024.pdf')
job = read_pdf('data/Data Scientist - Focus School Software _ Remote.pdf')

final_prompt = prompt.partial(sector=sector, resume=resume, job=job)
final_prompt

ChatPromptTemplate(input_variables=['history', 'input'], partial_variables={'sector': 'data science and analytics', 'resume': 'JAMES M. IRVING, PH.D. 8222 Spadderdock Way, Laurel, MD, 20724 | (518) 322-6750 | james.irving.phd@gmail.com |  LINKEDIN: james-irving-phd | GITHUB: https://github.com/jirvingphd  SUMMARY Innovative and enterprising data scientist. Experienced in applying advanced data science techniques to real-world problems, including statistical modeling, machine learning, and data visualization. Skilled at extracting valuable insights from complex datasets and supporting data-driven decision-making. Adept Python programmer with the demonstrated ability to rapidly master and apply new technologies. A highly analytical professional committed to continuous learning and leveraging the value of data to innovate and transform. Former neuroscience researcher with a solid foundation in science, experimental design, and cognitive neuroscience. COMPETENCIES • Data Analysis • Statist

In [26]:
model_type =  "gpt-3.5-turbo-0125"


llm = ChatOpenAI(temperature=.1, model=model_type)
llm_chain = ConversationChain(prompt=final_prompt, llm=llm, 
                              memory=ConversationBufferWindowMemory(memory_key='history',
                                                                    # return_messages=True
                                                                    ),
                                verbose=False, output_key="response")#,callbacks=[get_openai_callback()])
llm_chain

ConversationChain(memory=ConversationBufferWindowMemory(), prompt=ChatPromptTemplate(input_variables=['history', 'input'], partial_variables={'sector': 'data science and analytics', 'resume': 'JAMES M. IRVING, PH.D. 8222 Spadderdock Way, Laurel, MD, 20724 | (518) 322-6750 | james.irving.phd@gmail.com |  LINKEDIN: james-irving-phd | GITHUB: https://github.com/jirvingphd  SUMMARY Innovative and enterprising data scientist. Experienced in applying advanced data science techniques to real-world problems, including statistical modeling, machine learning, and data visualization. Skilled at extracting valuable insights from complex datasets and supporting data-driven decision-making. Adept Python programmer with the demonstrated ability to rapidly master and apply new technologies. A highly analytical professional committed to continuous learning and leveraging the value of data to innovate and transform. Former neuroscience researcher with a solid foundation in science, experimental design, 

In [27]:


def get_llm(model_type="gpt-3.5-turbo-0125", temperature=0.1,
            system_prompt_template_func= get_system_prompt_str,#verbose=False,
             verbose=False, sector="data science and analytics", resume='', job=''):
    """Version 2.0"""
    ## get prompt string
    system_prompt = system_prompt_template_func()
    final_promp_str = system_prompt + """
        Current conversation:
        {history}
        Human: {input}
        AI:"""
        
    final_prompt_template = ChatPromptTemplate.from_template(final_promp_str)
    final_prompt_template = final_prompt_template.partial(sector=sector, resume=resume, job=job)
        
    llm = ChatOpenAI(temperature=temperature, model=model_type)
    
    llm_chain = ConversationChain(prompt=final_prompt_template, 
                                  llm=llm, 
                                  memory=ConversationBufferWindowMemory(memory_key='history',
                                                                        human_prefix="Human",
                                                                        ai_prefix="AI",
                                                                        k=3),
                                  verbose=verbose, 
                                #   input_key="input",
                                  output_key="response")#,#callbacks=callbacks)
    
    return llm_chain


In [28]:
llm_chain_no_ctxt = get_llm()
llm_chain_no_ctxt


ConversationChain(memory=ConversationBufferWindowMemory(k=3), prompt=ChatPromptTemplate(input_variables=['history', 'input'], partial_variables={'sector': 'data science and analytics', 'resume': '', 'job': ''}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['history', 'input', 'job', 'resume', 'sector'], template=" You are a a specialized career coach for the {sector}, focused on delivering tailored, concise job application advice and practice.  You are proficient in resume analysis, cover letter guidance, and interview preparation, adapting to each user's unique requirements.  You maintain a professional, friendly tone, and encouraging tone, ensuring advice is efficient, clear, and easily understandable,  with the goal of aiding their career progression. Ask the user for their resume and job listing if not provided and they are needed to asnwer .\nUse the following context, if provided, to help answer the questions:\n\nHere is my resume:\n-------------\n {

In [29]:
llm_chain = get_llm(resume=resume, job=job)
llm_chain

ConversationChain(memory=ConversationBufferWindowMemory(k=3), prompt=ChatPromptTemplate(input_variables=['history', 'input'], partial_variables={'sector': 'data science and analytics', 'resume': 'JAMES M. IRVING, PH.D. 8222 Spadderdock Way, Laurel, MD, 20724 | (518) 322-6750 | james.irving.phd@gmail.com |  LINKEDIN: james-irving-phd | GITHUB: https://github.com/jirvingphd  SUMMARY Innovative and enterprising data scientist. Experienced in applying advanced data science techniques to real-world problems, including statistical modeling, machine learning, and data visualization. Skilled at extracting valuable insights from complex datasets and supporting data-driven decision-making. Adept Python programmer with the demonstrated ability to rapidly master and apply new technologies. A highly analytical professional committed to continuous learning and leveraging the value of data to innovate and transform. Former neuroscience researcher with a solid foundation in science, experimental desig

# Developing New Prompts

### Resume Comparison

In [30]:
# move this line to the task prompt
task_prompt_compare_resume = (" How does my resume compare against this job listing? "
"First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'. "
" Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements. "
" Finally, provide suggestions for what to address in a cover letter and wish me luck!"
" After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task.")

task_prompt_compare_resume

" How does my resume compare against this job listing? First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'.  Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements.  Finally, provide suggestions for what to address in a cover letter and wish me luck! After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task."

In [31]:
" How does my resume compare against this job listing? First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'.  Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements.  Finally, provide suggestions for what to address in a cover letter and wish me luck! After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task."

" How does my resume compare against this job listing? First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'.  Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements.  Finally, provide suggestions for what to address in a cover letter and wish me luck! After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task."

In [32]:
# Testing response when resuem and job not provided
output = llm_chain_no_ctxt.invoke({'input':task_prompt_compare_resume})
print(output['response'])

Great! Please provide me with your resume and the job listing so I can perform a detailed comparison for you. Once I have that information, I will categorize your resume's fit for the job and provide you with a breakdown of how it aligns with the job requirements. Let's get started!


In [33]:
# Testing response with context
output = llm_chain.invoke({'input':task_prompt_compare_resume})
# output = llm_chain.invoke({'input':task_prompt_compare_resume})
print(output['response'])

Based on your resume, your fit for the Data Scientist position at Focus School Software can be categorized as 'great.' Here is a detailed breakdown comparing your resume to the job listing:

Strengths:
1. Experience in data analysis, statistical modeling, machine learning, and data visualization align well with the job requirements.
2. Proficiency in Python programming, which is a key skill mentioned in the job listing.
3. Experience in working with large datasets and extracting valuable insights, which is crucial for the role.
4. Strong educational background in neuroscience and data science, showcasing a solid foundation for the position.
5. Experience in curriculum development and instructional delivery, demonstrating communication and teaching skills.

Weaknesses/Missing Requirements:
1. The job listing specifically mentions experience with Power BI Analytics and DAX, which are not explicitly mentioned in your resume. It would be beneficial to highlight any relevant experience or s

In [34]:
# output = llm_chain.invoke({'input':"Sure lets write the cover letter."})
# print(output['response'])

### Interview Prep

In [35]:
# move this line to the task prompt
task_prompt_interview_prep = (" Help me prep for an upcoming interview. "
"First, ask any questions that will help you understand which type of interview and job I am preparing for. "
"Next, ask if I want to do A)a guided practice interview where you explain what the questions are designed to test and provide feedback. "
" or B) a mock interview with feedback at the end of the mock interview. "
# "Cover both general interview questions as well as technical questions, depending on the type of job and type of interview. "
)

print(task_prompt_interview_prep)

 Help me prep for an upcoming interview. First, ask any questions that will help you understand which type of interview and job I am preparing for. Next, ask if I want to do A)a guided practice interview where you explain what the questions are designed to test and provide feedback.  or B) a mock interview with feedback at the end of the mock interview. 


In [37]:
llm_chain = get_llm()
output = llm_chain.invoke({'input':task_prompt_compare_resume})
print(output['response'])

Great! Please provide me with your resume and the job listing so I can compare them for you. Once I have both documents, I will categorize your resume's fit for the job and provide you with a detailed breakdown. Let's get started!


In [38]:
output = llm_chain.invoke({'input':task_prompt_interview_prep})
print(output['response'])

Great! I'd be happy to help you prep for your upcoming interview. Can you please provide me with some information about the type of interview and job you are preparing for? Once I have a better understanding, I can tailor my advice to best suit your needs. 

After that, would you prefer to do a guided practice interview where I explain what the questions are designed to test and provide feedback, or would you prefer to do a mock interview with feedback at the end? Let me know so we can get started!


In [39]:
output = llm_chain.invoke({'input':"The job is the one I uploaded to you earlier. The interview is a second round interview"})
print(output['response'])

Great, thank you for providing that information. Since this is a second round interview for the job you previously uploaded, I will tailor my advice accordingly. 

For a second round interview, it's important to showcase your specific skills and experiences that make you a strong fit for the role. Be prepared to discuss in more detail how your background aligns with the job requirements and how you can contribute to the company's success.

Would you like to do a guided practice interview where I explain what the questions are designed to test and provide feedback, or would you prefer to do a mock interview with feedback at the end? Let me know so we can get started!


## New Task Prompts

> Note (04/15/24): adding new config folder with json file of prompts.

In [41]:
task_prompt_compare_resume = (" How does my resume compare against this job listing? "
"First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'. "
" Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements. "
" Finally, provide suggestions for what to address in a cover letter and wish me luck!"
" After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task.")

task_prompt_compare_resume

" How does my resume compare against this job listing? First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'.  Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements.  Finally, provide suggestions for what to address in a cover letter and wish me luck! After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task."

In [42]:
task_prompt_interview_prep = (" Help me prep for an upcoming interview. "
"First, ask any questions that will help you understand which type of interview and job I am preparing for. "
"Next, ask if I want to do A)a guided practice interview where you explain what the questions are designed to test and provide feedback. "
" or B) a mock interview with feedback at the end of the mock interview. "
# "Cover both general interview questions as well as technical questions, depending on the type of job and type of interview. "
)

print(task_prompt_interview_prep)

 Help me prep for an upcoming interview. First, ask any questions that will help you understand which type of interview and job I am preparing for. Next, ask if I want to do A)a guided practice interview where you explain what the questions are designed to test and provide feedback.  or B) a mock interview with feedback at the end of the mock interview. 


In [50]:
task_prompt_cover_letter =(" Write a cover letter for this job using my resume. Use a memorable opening sentence that is very enthusiastic about the job. "  
" Highlight my strengths and address any obvious missing qualifications.  ") 
task_prompt_cover_letter



' Write a cover letter for this job using my resume. Use a memorable opening sentence that is very enthusiastic about the job.  Highlight my strengths and address any obvious missing qualifications.  '

In [51]:
# task_craft_cover_letter = (" "
#                            " "
#                            )

In [52]:
prompts = {"Compare Resuem vs. Job Listing":task_prompt_compare_resume,
               "Interivew Prep": task_prompt_interview_prep,
               "Draft a Cover Letter": task_prompt_cover_letter}
prompts

{'Compare Resuem vs. Job Listing': " How does my resume compare against this job listing? First, categorize my resume's fit for the job as 'perfect,' 'great,' 'good,' or 'poor'.  Next, provide a list with the detailed break down comparing my resume vs. the job. Highlight any weaknesses or missing requirements.  Finally, provide suggestions for what to address in a cover letter and wish me luck! After performing the analysis, ask if the user wants to continue with a cover letter or move on to another task.",
 'Interivew Prep': ' Help me prep for an upcoming interview. First, ask any questions that will help you understand which type of interview and job I am preparing for. Next, ask if I want to do A)a guided practice interview where you explain what the questions are designed to test and provide feedback.  or B) a mock interview with feedback at the end of the mock interview. ',
 'Draft a Cover Letter': ' Write a cover letter for this job using my resume. Use a memorable opening senten

In [53]:
## create the config folder and json file
import os, json
config_folder = "config/"
os.makedirs(config_folder, exist_ok=True)

# save the config file for prompts.
prompt_config_file  = os.path.join(config_folder, "prompt_config.json")
prompt_config_file

if not os.path.exists(prompt_config_file):
    with open(prompt_config_file, 'w') as f:
        f.write(json.dumps(prompts))

In [54]:
with open(prompt_config_file, 'r') as f:
    loaded_prompts = json.load(f)
loaded_prompts.keys()

dict_keys(['Compare Resuem vs. Job Listing', 'Interivew Prep', 'Draft a Cover Letter'])